In [ ]:
from ltlcross_runner import param_runner, LtlcrossRunner
from tools_hier import get_tools, tool_order
from evaluation_utils import split_cols, sort_by_tools, to_tuples

## Get the edge where all tools succeed

In [ ]:
def param_get_edge_n(self,only_timeout=True,tool_subset=None):
    if tool_subset is None:
        tool_subset = self.tools.keys()
    v = self.values.states[list(tool_subset)]
    max_n = len(v.dropna())
    if only_timeout:
        es = self.exit_status.iloc[max_n]
        while len(es[es == 'timeout']) == 0:
            max_n += 1
            es = r.exit_status.iloc[max_n]
    return max_n

## Compute n for each tool

In [ ]:
def param_get_max_n(self,tool):
    v = self.values.states[tool]
    max_n = len(v.dropna())
    return max_n

## get_nX_states

In [ ]:
def param_get_value_for(self,tool,n,col='states'):
    v = self.values[col][tool]
    return v.iloc[n-1]

## Prepare table

In [ ]:
import pandas as pd

In [ ]:
def my_highlight_best_min(data):
    """highlight the minimum of a row."""
    return ['background-color: #3db709' if m == data.min() else ''
            for m in data]

def my_highlight_best_max(data):
    """highlight the maxinum of a row."""
    return ['background-color: #3db709' if m == data.max() else ''
            for m in data]

def my_highlight_max(data):
    """highlight the maximum of a row, and also highlight values that are not far away."""
    m = data.max()
    is_max = data == m
    is_good = (data >= (m/1.05)) | (data >= m-0.005)
    return ['background-color: #ff5050' if m else
            'background-color: #ff9999' if g else ''
            for m,g in zip(is_max, is_good)]

In [ ]:
#maxn = "max $n$"
maxn = "max"
n_str = '$n = {}$'
ne_str = '$n_e = {}$'
nt_str = '$n_t = {}$'
def compute_param(self,col='states',tool_subset=None):
    if tool_subset is None:
        tool_subset = self.tools.keys()
    nt = param_get_edge_n(self,tool_subset=tool_subset)
    ne = param_get_edge_n(self,False,tool_subset=tool_subset)
    
    if ne == nt:
        n_i = n_str.format(ne)
        cols = [n_i,maxn]
    else:
        ne_i = ne_str.format(ne)
        nt_i = nt_str.format(nt)
        cols = [ne_i,nt_i,maxn]
    idx = pd.Index(data=cols)
    df = pd.DataFrame(columns=tool_subset,index=idx)

    v = self.values[col]
    if ne == nt:
        df.loc[n_i] = v[df.columns].iloc[ne-1]
    else:
        df.loc[nt_i] = v[df.columns].iloc[nt-1]
        df.loc[ne_i] = v[df.columns].iloc[ne-1]
    for tool in tool_subset:
        df.loc[maxn].loc[tool] = float(param_get_max_n(self,tool))
    #tmp = df.copy()
    return df#tmp.style.apply(my_highlight_min, axis=1)

In [ ]:
ltlgux = ['gh-e','gh-c1','gh-c2','gh-q','gh-r',
          'gh-s','gh-u','gh-u2','ms-phi-r',
          'ms-phi-s','and-fg','or-fg','go-theta']
full = ['kr-n','kr-nlogn','ms-phi-h']

In [ ]:
','.join(ltlgux)

In [ ]:
data = []
for f in ltlgux:
    r = param_runner(f,get_tools('ltl-gux'))
    r.parse_results()
    data.append(compute_param(r))
    
for f in full:
    r = param_runner(f,get_tools('full'))
    r.parse_results()
    data.append(compute_param(r))

res = pd.concat(data,keys=ltlgux+full)

In [ ]:
def highlight_best(res):  
    # Choose all rows for numbers of states
    a = res.index.levels[1]
    for_min = a.delete(a.get_loc(maxn))

    idx = pd.IndexSlice
    # Color the best values
    return res.style\
        .apply(my_highlight_best_min, axis=1,subset=idx[idx[:,for_min],:])\
        .apply(my_highlight_best_max, axis=1,subset=idx[idx[:,maxn],:])

In [ ]:
h = highlight_best(res)
h

In [ ]:
split = split_cols(res)
split = sort_by_tools(split,tool_order,axis=1)
highlight_best(split)

In [ ]:
highlight_best(split[['R3','R4']])

In [ ]:
def param_to_latex(res,file='param.tex',transpose=False):
    if transpose:
        res = res.T
        col_f = 'rrr' + 'r'*len(res.columns)
    else:
        col_f = 'cc' + 'r'*len(res.columns)
    res.to_latex(buf=open(file,'w'),multirow=True,escape=False,na_rep='---',float_format=lambda x: '$' + '%0.0f' % x + '$',column_format=col_f,multicolumn_format='c')

In [ ]:
param_to_latex(split,transpose=False)

In [ ]:
def multirow(s):
    return '\multirow[t]{2}{*}{'+s+'}'

In [ ]:
idx = pd.IndexSlice
split = split_cols(res)
highlight_best(split.loc(axis=1)[idx[:,:,['DTPA','NBA.DTPA','TGBA.DTPA']]])